In [1]:
import os
data_dir = os.path.expanduser("~/Google Drive/Shared drives/Data")
dataset = "bioasq"
data_path = f'{data_dir}/BEIR/{dataset}'

In [21]:
from vectorspace.utils import json_load
from beir.datasets.data_loader import GenericDataLoader
_, queries, qrels = GenericDataLoader(data_path).load(split="test")
bm25_results = json_load(f"{data_dir}/BEIR/results_{dataset}_bm25.json")
bm25_results_100 = json_load(f"{data_dir}/BEIR/results_{dataset}_bm25_100.json")

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
from vectorspace.store import Store as ESDict
from elasticsearch import Elasticsearch
corpus = ESDict(Elasticsearch(['http://localhost:9200']), dataset)

In [8]:
#doc_ids = [doc_id for doc_ids in bm25_results.values() for doc_id in list(doc_ids)[:1000]]
#reduced_corpus = {}
#for doc_id, doc in corpus.mget(set(doc_ids), _source=['title', 'text']):
#    reduced_corpus[doc_id] = {
#        'title': doc['title'] or '',
#        'text': doc['text'] or ''
#    }

In [23]:
doc_ids = [doc_id for doc_ids in bm25_results_100.values() for doc_id in list(doc_ids)[:1000]]
reduced_corpus_100 = {}
for doc_id, doc in corpus.mget(set(doc_ids), _source=['title', 'text']):
    reduced_corpus_100[doc_id] = {
        'title': doc['title'] or '',
        'text': doc['text'] or ''
    }

In [24]:
len(reduced_corpus_100)

86980

In [25]:
from vectorspace.utils import json_dump
json_dump(reduced_corpus_100, f"{data_path}/reduced_corpus_100.json")

In [20]:
from vectorspace.utils import json_load
reduced_corpus = json_load(f"{data_path}/reduced_corpus.json")

In [33]:
for query_id, docs in bm25_results.items():
    query = queries[query_id]
    print(query)
    print()
    for doc_id in list(docs)[:5]:
        doc = corpus[doc_id]
        print(doc['title'])
        print(doc_id in qrels[query_id])
        print()
    print()

In [27]:
list(reduced_corpus.items())[:10]

[('29434076',
  {'title': 'Differences in Gut Microbiota Profiles between Autoimmune Pancreatitis and Chronic Pancreatitis.',
   'text': 'Host-derived factors alter gut microenvironment, and changes in gut microbiota also affect biological functions of host. Alterations of gut microbiota have been reported in a wide variety of diseases, but the whole picture of alterations in pancreatic diseases remains to be clarified. In particular, the gut microbiota may be affected by malnutrition or impaired exocrine pancreas function that is associated with pancreatic diseases. We here conducted comprehensive analysis of gut microbiota in patients with type 1 autoimmune pancreatitis (AIP), a pancreatic manifestation of the systemic IgG4-related disease, and chronic pancreatitis (CP). The two diseases were selected, because altered immune reactions in AIP and/or long-standing malnutrition in CP may influence the gut microbiota. Fecal samples were obtained from 12 patients with AIP before the stero